In [1]:
import os
import sys
from nltk.corpus import stopwords
import re
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession, SQLContext
import os
import pyspark.sql.functions as f
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

22/04/15 10:05:02 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
22/04/15 10:05:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/15 10:05:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = SparkSession.builder.appName('App Name').master("local[*]").getOrCreate()
sqlContext = SQLContext(sparkContext=sc.sparkContext, sparkSession=sc)

In [4]:
classics = sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load(os.path.realpath("data/fanfics_raw.csv"))

In [5]:
classics.show()

+--------------------+--------------------+--------------------+
|                  id|                text|                type|
+--------------------+--------------------+--------------------+
|             1536152|           Chapter 1|                null|
|Going back was th...| after a case lik...| I’d be moved to ...|
|           Chapter 2|                null|                null|
|It was during my ...| Sophia was no lo...| especially since...|
|           Chapter 3|                null|                null|
|Following Sophia ...| I ducked in an e...| waiting under a ...|
|           Chapter 4|                null|                null|
|Aegis breaking in...| but when they ca...| barely managing ...|
|           Chapter 5|                null|                null|
|I hated coming to...|                sure| but everyone her...|
|           Chapter 6|                null|                null|
|             Somehow| this time was ev...| my instincts wer...|
|           Chapter 7|   

In [6]:
import string

In [7]:
classics_df = classics.withColumn('line', f.explode(f.split(f.col('text'), '\n')))
classics_df.show()

+--------------------+--------------------+--------------------+--------------------+
|                  id|                text|                type|                line|
+--------------------+--------------------+--------------------+--------------------+
|             1536152|           Chapter 1|                null|           Chapter 1|
|Going back was th...| after a case lik...| I’d be moved to ...| after a case lik...|
|It was during my ...| Sophia was no lo...| especially since...| Sophia was no lo...|
|Following Sophia ...| I ducked in an e...| waiting under a ...| I ducked in an e...|
|Aegis breaking in...| but when they ca...| barely managing ...| but when they ca...|
|I hated coming to...|                sure| but everyone her...|                sure|
|             Somehow| this time was ev...| my instincts wer...| this time was ev...|
|The restaurant Li...| just off the Boa...| and without thos...| just off the Boa...|
|I managed to snea...| I tested for spe...| there were

In [8]:
classics = classics_df.rdd.map(lambda x : x['line']).filter(lambda x: x is not None)
StopWords = stopwords.words("english")
tokens = classics                                                  \
    .map( lambda document: document.strip().lower())               \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])    \
    .zipWithIndex()

In [9]:
from pyspark.sql.functions import udf, col, StringType, column
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, IDF
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer
df_txts = sqlContext.createDataFrame(tokens, ["list_of_words",'index'])# TF
cv = CountVectorizer(inputCol="list_of_words", outputCol="raw_features", vocabSize=5000, minDF=1.0)
cvmodel = cv.fit(df_txts)
result_cv = cvmodel.transform(df_txts)
# IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv) 

In [10]:
from pyspark.sql.functions import udf, col, StringType, column
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer
df_txts = sqlContext.createDataFrame(tokens, ["list_of_words",'index'])# TF
cv = CountVectorizer(inputCol="list_of_words", outputCol="raw_features", vocabSize=5000, minDF=1.0)
cvmodel = cv.fit(df_txts)
result_cv = cvmodel.transform(df_txts)
# IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv) 

In [11]:
num_topics = 10
max_iterations = 100
lda = LDA(k=num_topics, maxIter=max_iterations)
lda_model = lda.fit(result_tfidf[['index','features']])

22/04/15 10:05:19 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/04/15 10:05:19 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [12]:
classics_df.printSchema()
classics_df.show()

root
 |-- id: string (nullable = true)
 |-- text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- line: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+
|                  id|                text|                type|                line|
+--------------------+--------------------+--------------------+--------------------+
|             1536152|           Chapter 1|                null|           Chapter 1|
|Going back was th...| after a case lik...| I’d be moved to ...| after a case lik...|
|It was during my ...| Sophia was no lo...| especially since...| Sophia was no lo...|
|Following Sophia ...| I ducked in an e...| waiting under a ...| I ducked in an e...|
|Aegis breaking in...| but when they ca...| barely managing ...| but when they ca...|
|I hated coming to...|                sure| but everyone her...|                sure|
|             Somehow| this time was ev...| my instincts wer...| this time was

In [13]:
lda_model.describeTopics().show()

+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[14, 139, 189, 17...|[0.02141250317791...|
|    1|[66, 28, 31, 113,...|[0.01756785798549...|
|    2|[5, 38, 7, 56, 24...|[0.02257030944954...|
|    3|[30, 33, 37, 44, ...|[0.02118311042827...|
|    4|[23, 32, 51, 47, ...|[0.02362122768140...|
|    5|[3, 16, 10, 17, 2...|[0.04733502409303...|
|    6|[20, 19, 71, 65, ...|[0.02669326733037...|
|    7|[11, 39, 29, 1, 2...|[0.02726499944641...|
|    8|[35, 107, 59, 146...|[0.02257721499493...|
|    9|[0, 22, 48, 82, 6...|[0.04670586419601...|
+-----+--------------------+--------------------+



/opt/spark/python/lib/pyspark.zip/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.


In [14]:
from pyspark.sql.types import *
vocab = cvmodel.vocabulary
vocab_broadcast = sc.sparkContext.broadcast(vocab)

#creating LDA model
ldatopics = lda_model.describeTopics()

def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])

    return words

udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))

ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))

In [15]:
ldatopics_mapped.show()

+-----+--------------------+--------------------+--------------------+
|topic|         termIndices|         termWeights|          topic_desc|
+-----+--------------------+--------------------+--------------------+
|    0|[14, 139, 189, 17...|[0.02141250317791...|[really, feeling,...|
|    1|[66, 28, 31, 113,...|[0.01756785798549...|[another, eyes, f...|
|    2|[5, 38, 7, 56, 24...|[0.02257030944954...|[know, though, ev...|
|    3|[30, 33, 37, 44, ...|[0.02118311042827...|[chloe, looking, ...|
|    4|[23, 32, 51, 47, ...|[0.02362122768140...|[make, sure, alre...|
|    5|[3, 16, 10, 17, 2...|[0.04733502409303...|[says, beca, quin...|
|    6|[20, 19, 71, 65, ...|[0.02669326733037...|[elizabeth, still...|
|    7|[11, 39, 29, 1, 2...|[0.02726499944641...|[think, well, rig...|
|    8|[35, 107, 59, 146...|[0.02257721499493...|[look, turned, ha...|
|    9|[0, 22, 48, 82, 6...|[0.04670586419601...|[said, darcy, nee...|
+-----+--------------------+--------------------+--------------------+



In [16]:
from pyspark.sql.functions import concat_ws, split
import pyspark.sql.functions as f
lda_topics_mapped = ldatopics_mapped.withColumn("topic_desc", concat_ws(" ", ldatopics_mapped.topic_desc).alias("topic_desc_str"))
lda_topics_mapped = lda_topics_mapped.withColumn("termWeights", concat_ws(" ", ldatopics_mapped.topic_desc).alias("topic_desc_str"))
lda_topics_mapped.show()

AnalysisException: Resolved attribute(s) topic_desc#190 missing from topic#183,termIndices#184,termWeights#185,topic_desc#214 in operator !Project [topic#183, termIndices#184, concat_ws( , topic_desc#190) AS termWeights#220, topic_desc#214]. Attribute(s) with the same name appear in the operation: topic_desc. Please check if the right attribute(s) are used.;
!Project [topic#183, termIndices#184, concat_ws( , topic_desc#190) AS termWeights#220, topic_desc#214]
+- Project [topic#183, termIndices#184, termWeights#185, concat_ws( , topic_desc#190) AS topic_desc#214]
   +- Project [topic#183, termIndices#184, termWeights#185, map_termID_to_Word(termIndices#184) AS topic_desc#190]
      +- Project [_1#177 AS topic#183, _2#178 AS termIndices#184, _3#179 AS termWeights#185]
         +- LocalRelation [_1#177, _2#178, _3#179]


In [ ]:
df = ldatopics_mapped.withColumn("new", f.arrays_zip("topic_desc", "termWeights"))\
       .withColumn("new", f.explode("new")) \
        .select("topic", f.col("new.topic_desc").alias("words"), f.col("new.termWeights").alias("termWeights"))

In [ ]:
df.show()

In [ ]:
import pyspark.sql.functions as F
from collections import ChainMap
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud(background_color="white")

words = dict(ChainMap(*df.select(F.create_map('words', 'termWeights')).rdd.map(lambda x: x[0]).collect()))
# {'scorbutically': 1.76, 'punta': 1.76, 'detail': 1.789, 'lafayette': 1.8, 'maya': 1.854, 'prostate': 1.854, 'quot': 1.856, 'mark': 1.949, 'elite': 1.988, 'trade': 2.012, 'write': 2.083}

plt.imshow(wordcloud.generate_from_frequencies(words))

plt.show()
